In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

### Tetgen multi-domain meshing proof of concept

In [2]:
from itertools import islice
from collections import namedtuple
import numpy as np

Tetgen file formats:
http://wias-berlin.de/software/tetgen/1.5/doc/manual/manual006.html#ff_poly

The data below constructs a rectangle within a cube. The rectangle cuts from one end to the other end of the cube, simulating a 'pore'. The inner rectangle should be handled as a different domain by the algorithm. The goal is to develop a workflow that demonstrates the multi-domain meshing.

In [147]:
from nanomesh.mesh_container import TriangleMesh

vertices = np.array([
  [0.0, 0.0, 0.0],  # A
  [4.0, 0.0, 0.0],  # B
  [4.0, 4.0, 0.0],  # C
  [0.0, 4.0, 0.0],  # D
  [0.0, 0.0, 4.0],  # E
  [4.0, 0.0, 4.0],  # F
  [4.0, 4.0, 4.0],  # G
  [0.0, 4.0, 4.0],  # H
    
  [1.0, 1.0, 0.0],  # a
  [3.0, 1.0, 0.0],  # b
  [3.0, 3.0, 0.0],  # c
  [1.0, 3.0, 0.0],  # d
  [1.0, 1.0, 4.0],  # e
  [3.0, 1.0, 4.0],  # f
  [3.0, 3.0, 4.0],  # g
  [1.0, 3.0, 4.0],  # h
])

faces = np.array([
    [0, 11, 8 ],
    [1, 8,  9 ],
    [2, 9,  10],
    [3, 10, 11],
    [0, 8,  1 ],
    [1, 9,  2 ],
    [2, 10, 3 ],
    [3, 11, 0 ],

    [0, 1, 5],
    [5, 4, 0],
    [1, 2, 6],
    [6, 5, 1],
    [3, 2, 6],
    [6, 7, 3],
    [0, 3, 7],
    [7, 4, 0],

    [4, 15,  12 ],
    [5, 12,  13 ],
    [6, 13,  14],
    [7, 14,  15],
    [4, 12,  5 ],
    [5, 13,  6 ],
    [6, 14,  7 ],
    [7, 15,  4 ],
    
    [8,  9,  10],
    [10, 11, 8 ],
    [8,  9,  13],
    [13, 12, 8 ],
    [9,  10, 14],
    [14, 13, 9 ],
    [11, 10, 14],
    [14, 15, 11],
    [8,  11, 15],
    [15, 12, 8 ],
    [13, 14, 15],
    [15, 12, 13],
])

triangles = TriangleMesh(vertices=vertices, faces=faces)
triangles.plot_itk()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Region markers assigns attributes to tetrahedra in different regions.

In [148]:
from nanomesh import tetgen

region_markers = {
  10: [0.5, 0.5, 0.5],
  20: [2.0, 2.0, 2.0],   
}

tetras = triangles.tetrahedralize(region_markers=region_markers)
tetras.metadata

{'tetgen.ref': array([10, 10, 20, ..., 10, 10, 10]),
 'labels': array([0, 0, 0, ..., 0, 0, 0])}

In [149]:
tetras.metadata['labels2'] = tetras.metadata.pop('tetgen.ref')

ITK is a bit picky about the name of the metadata, and will sometimes not plot it. Use `tetview` to view the `.ele` files.

https://wias-berlin.de/software/tetgen/tetview.html

In [150]:
tetras.plot_itk()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…